In [ ]:
from google.colab import drive
import os

# 1. Montar o Drive
drive.mount('/content/drive')

# 2. Instalar bibliotecas
!pip install geobr geopandas matplotlib pandas shapely scikit-learn

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
# Lista os arquivos na pasta do Drive
pasta = '/content/drive/MyDrive//Projetos/Projeto - Logística/'
try:
    print("Arquivos encontrados na pasta:")
    print(os.listdir(pasta))
except Exception as e:
    print("Erro ao acessar a pasta. Verifique se o nome está correto!")
    print(e)

Arquivos encontrados na pasta:
['populacao_curitiba.csv', 'SIST_VIARIO_CLASSIFICADO.sbx', 'SIST_VIARIO_CLASSIFICADO.prj', 'SIST_VIARIO_CLASSIFICADO.sbn', 'SRE_2021_UTM_SIRGAS2000_22S_LN.shp', 'SRE_2021_UTM_SIRGAS2000_22S_LN.qmd', 'SRE_2021_UTM_SIRGAS2000_22S_LN.dbf', 'SRE_2021_UTM_SIRGAS2000_22S_LN.cpg', 'SRE_2021_UTM_SIRGAS2000_22S_LN.shx', 'OUC_LV_PERIMETRO_SETORES.shx', 'OUC_LV_PERIMETRO_SETORES.sbx', 'OUC_LV_PERIMETRO_SETORES.prj', 'OUC_LV_PERIMETRO_SETORES.dbf', 'OUC_LV_PERIMETRO_SETORES.sbn', 'OUC_LV_PERIMETRO_SETORES.shp', 'SIST_VIARIO_CLASSIFICADO.shp', 'SIST_VIARIO_CLASSIFICADO.shp.xml', 'SIST_VIARIO_CLASSIFICADO.dbf', 'SIST_VIARIO_CLASSIFICADO.shx', 'ZONA_HISTORICA.shx', 'ZONA_HISTORICA.sbn', 'ZONA_HISTORICA.shp', 'SE_DE_PEDESTRE.dbf', 'SE_DE_PEDESTRE.shp', 'SETOR_DESPORTIVO_E_DE_ESTADIOS.dbf', 'SE_DE_PEDESTRE.sbx', 'ZONA_HISTORICA.prj', 'SETOR_DESPORTIVO_E_DE_ESTADIOS.sbx', 'SETOR_DESPORTIVO_E_DE_ESTADIOS.shp', 'ZONA_HISTORICA.dbf', 'SE_DE_PEDESTRE.lyr', 'ZONEAMENTO.sbn', 'S

In [ ]:
import geobr
import pandas as pd
import geopandas as gpd

# Define o caminho para a sua pasta no Drive
# Certifique-se de que o nome da pasta no seu Drive é exatamente este
PATH = r'/content/drive/MyDrive/Projetos/Projeto - Logística/'

print("⏳ Baixando malha de Curitiba e lendo CSV de população...")

# 1. Baixar setores de Curitiba (2022)
setores = geobr.read_census_tract(code_tract=4106902, year=2022)

# 2. Ler o CSV que está no seu Drive
df_pop = pd.read_csv(PATH + 'populacao_curitiba.csv', sep=';', dtype=str, encoding='iso-8859-1')

# 3. Limpeza e Cruzamento (Merge)
df_pop['code_tract'] = df_pop['CD_SETOR'].str.extract('(\d+)')[0]
setores['code_tract'] = setores['code_tract'].astype(str).str.extract('(\d+)')[0]

curitiba_final = setores.merge(df_pop[['code_tract', 'v0001', 'NM_BAIRRO']], on='code_tract', how='left')
curitiba_final['v0001'] = pd.to_numeric(curitiba_final['v0001'], errors='coerce').fillna(0)

print(f"✅ Sucesso! {len(curitiba_final)} setores processados.")

<>:18: SyntaxWarning: invalid escape sequence '\d'
<>:19: SyntaxWarning: invalid escape sequence '\d'
<>:18: SyntaxWarning: invalid escape sequence '\d'
<>:19: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-4051271230.py:18: SyntaxWarning: invalid escape sequence '\d'
  df_pop['code_tract'] = df_pop['CD_SETOR'].str.extract('(\d+)')[0]
/tmp/ipython-input-4051271230.py:19: SyntaxWarning: invalid escape sequence '\d'
  setores['code_tract'] = setores['code_tract'].astype(str).str.extract('(\d+)')[0]


⏳ Baixando malha de Curitiba e lendo CSV de população...
✅ Sucesso! 3190 setores processados.


In [ ]:
import geopandas as gpd
PATH = r'/content/drive/MyDrive/Projetos/Projeto - Logística/'

# Carrega apenas o cabeçalho das vias para conferir os nomes
vias_teste = gpd.read_file(PATH + 'SIST_VIARIO_CLASSIFICADO.shp', rows=5)
print("As colunas do seu arquivo de vias são:")
print(vias_teste.columns.tolist())

As colunas do seu arquivo de vias são:
['OBJECTID', 'CODVIA', 'NMVIA', 'STATUS', 'SIST_VIARI', 'SIST_VIA_1', 'DATA_ATUAL', 'DATA_VERSA', 'SHAPE_Leng', 'geometry']


In [ ]:
from sklearn.cluster import KMeans
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import numpy as np
import os

# 1. Configurações de Projeção (UTM para Curitiba)
CRS_UTM = "EPSG:31982"
curitiba_utm = curitiba_final.to_crs(CRS_UTM).copy()
curitiba_utm['x'] = curitiba_utm.geometry.centroid.x
curitiba_utm['y'] = curitiba_utm.geometry.centroid.y

print("🛰️ Calculando Hubs Estratégicos com Links do Maps...")

# 2. Carregar Camadas de Suporte
vias_todas = gpd.read_file(PATH + 'SIST_VIARIO_CLASSIFICADO.shp').to_crs(CRS_UTM)
rodovias = gpd.read_file(PATH + 'SRE_2021_UTM_SIRGAS2000_22S_LN.shp').to_crs(CRS_UTM)

path_lv = PATH + 'OUC_LV_PERIMETRO_SETORES.shp' if os.path.exists(PATH + 'OUC_LV_PERIMETRO_SETORES.shp') else PATH + 'OUC_LV_PERIMETRO.shp'
lv = gpd.read_file(path_lv).to_crs(CRS_UTM)

# 3. Criar Zona de Atração (1km das Rodovias/Linha Verde)
eixos_principais = pd.concat([rodovias.geometry, lv.geometry]).union_all()
zona_permitida = eixos_principais.buffer(1000)

# 4. Filtrar e Rodar IA
mask = (curitiba_utm.geometry.intersects(zona_permitida)) & (curitiba_utm['v0001'] > 0)
dados_ia = curitiba_utm.loc[mask, ['x', 'y']].copy()
pesos = curitiba_utm.loc[mask, 'v0001']

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
kmeans.fit(dados_ia, sample_weight=pesos)

# 5. Criar GeoDataFrame dos Hubs
hubs = gpd.GeoDataFrame(geometry=[Point(x, y) for x, y in kmeans.cluster_centers_], crs=CRS_UTM)

# 6. Identificar Referências (Bairro e Eixo Logístico)
def identificar_info(p):
    bairro_sel = curitiba_utm[curitiba_utm.intersects(p)]['NM_BAIRRO']
    bairro = bairro_sel.iloc[0] if not bairro_sel.empty else "N/A"

    # Eixo Logístico Principal (ESTRUTURAL/RODOVIA/VIA RÁPIDA)
    vias_p = vias_todas[vias_todas['SIST_VIARI'].str.contains('ESTRUTURAL|RODOVIA|VIA RÁPIDA', na=False, case=False)]
    if not vias_p.empty:
        eixo = vias_p.loc[vias_p.distance(p).idxmin(), 'NMVIA']
    else:
        eixo = "Eixo Principal Próximo"

    return bairro, eixo

detalhes = hubs.geometry.apply(identificar_info)
hubs[['BAIRRO', 'EIXO_PRINC']] = pd.DataFrame(detalhes.tolist(), index=hubs.index)

# 7. Gerar Coordenadas e Links Oficiais do Google Maps
hubs_4326 = hubs.to_crs(epsg=4326)
hubs['LAT'] = hubs_4326.geometry.y
hubs['LONG'] = hubs_4326.geometry.x
# Formato universal do link do Google Maps para coordenadas
hubs['LINK_MAPS'] = "https://www.google.com/maps?q=" + hubs['LAT'].astype(str) + "," + hubs['LONG'].astype(str)

# 8. Salvar arquivo para o QGIS
hubs.to_file(PATH + "RESULTADO_HUBS_LOGISTICOS.shp")

print("\n🏆 RESULTADOS FINAIS:")
for i, row in hubs.iterrows():
    print(f"\nHub {i}: {row['BAIRRO']} | Referência: {row['EIXO_PRINC']}")
    print(f"Link: {row['LINK_MAPS']}")

print("\n✅ Arquivo 'RESULTADO_HUBS_LOGISTICOS.shp' gerado com sucesso!")

🛰️ Calculando Hubs Estratégicos com Links do Maps...

🏆 RESULTADOS FINAIS:

Hub 0: Santo Inácio | Referência: Eixo Principal Próximo
Link: https://www.google.com/maps?q=-25.425691778720246,-49.34333072298419

Hub 1: Capão Raso | Referência: Eixo Principal Próximo
Link: https://www.google.com/maps?q=-25.501178809221432,-49.30598681889207

Hub 2: Jardim das Américas | Referência: Eixo Principal Próximo
Link: https://www.google.com/maps?q=-25.461276290320964,-49.2302565456274

Hub 3: Bacacheri | Referência: Eixo Principal Próximo
Link: https://www.google.com/maps?q=-25.387616610485157,-49.228094030913326

Hub 4: Tatuquara | Referência: Eixo Principal Próximo
Link: https://www.google.com/maps?q=-25.555248558873384,-49.29524405466101

✅ Arquivo 'RESULTADO_HUBS_LOGISTICOS.shp' gerado com sucesso!


In [ ]:
# EXECUTE ESTE BLOCO PARA GERAR AS COORDENADAS PARA O DIAGNÓSTICO
print("📍 COORDENADAS DOS HUBS PARA ANÁLISE:")
for i, row in hubs.iterrows():
    print(f"Hub {i}: {row['BAIRRO']} | Lat: {row['LAT']}, Long: {row['LONG']}")

📍 COORDENADAS DOS HUBS PARA ANÁLISE:
Hub 0: Santo Inácio | Lat: -25.425691778720246, Long: -49.34333072298419
Hub 1: Capão Raso | Lat: -25.501178809221432, Long: -49.30598681889207
Hub 2: Jardim das Américas | Lat: -25.461276290320964, Long: -49.2302565456274
Hub 3: Bacacheri | Lat: -25.387616610485157, Long: -49.228094030913326
Hub 4: Tatuquara | Lat: -25.555248558873384, Long: -49.29524405466101
